# Курсовая ПГП

# Вариант 2

Задача: Есть два больших массива с координатами широты и долготы. Требуется получить количество точек в первом массиве, координаты которых в пределах 1 км ко второму массиву. Итак, если у нас есть два массива (n x 2 и k x 2, где n > k), результирующий массив должен быть массивом счетчиков размерности n, представляющих количество раз, когда точка в первом массиве n находится в пределах 1 км от точки в массиве k.

Однако в случае n = 10 000 000 и k = 1 000 000 пришлось бы вычислять 10 000 000 x 1 000 000 или 10 000 000 000 000 расстояний, что очень долго. Поэтому будет использоваться Numba для оптимизации процесса.

In [21]:
import math
import numpy as np

from numba import cuda, jit, prange, vectorize, guvectorize
from sys import getsizeof
from multiprocessing import cpu_count, Pool

In [22]:
n = 1_000
k = 1_000

coord1 = np.zeros((n, 2), dtype=np.float32)
coord2 = np.zeros((k, 2), dtype=np.float32)

coord1[:,0] = np.random.uniform(-90, 90, n).astype(np.float32)
coord1[:,1] = np.random.uniform(-180, 180, n).astype(np.float32)
coord2[:,0] = np.random.uniform(-90, 90, k).astype(np.float32)
coord2[:,1] = np.random.uniform(-180, 180, k).astype(np.float32)

coord1 = np.sort(coord1,axis=0)
coord2 = np.sort(coord2,axis=0)

Перед использование Numba, просмотрим скорость работы готовой библиотеки для задач с расстоянием. В функции также добавлен фильтр, чтобы исключить точки, которые слишком далеко, чтобы стоило вычислять их  расстояние.

In [41]:
from geopy.distance import great_circle

In [25]:
%timeit great_circle(coord1[0], coord2[0])

19.6 µs ± 759 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [26]:
def get_nearby_py(coord1, coord2, max_dist):
    out = []
    lat_filter = max_dist / 100
    for lat,lng in coord1:
        ct = 0
        for lat2,lng2 in coord2:
            if np.abs(lat - lat2) < lat_filter:
                if great_circle((lat,lng),(lat2,lng2)).km < max_dist:
                    ct += 1
        out.append(ct)
    return out

In [40]:
_t_py = %timeit -o get_nearby_py(coord1, coord2, 1.0)

5.39 s ± 49.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
est_time = lambda x : print( 'Примерно {} часов, чтобы перебрать полный датасет.'.format( round(1e13/(n*k) * x / 3600, 2) ) )
est_time(_t_py.average)

Примерно 14967.12 часов, чтобы перебрать полный датасет.


In [36]:
@cuda.jit(device=True)
def haversine_cuda(s_lat,s_lng,e_lat,e_lng):
    R = 6373.0

    s_lat = s_lat * math.pi / 180                     
    s_lng = s_lng * math.pi / 180 
    e_lat = e_lat * math.pi / 180                    
    e_lng = e_lng * math.pi / 180 

    d = math.sin((e_lat - s_lat)/2)**2 + math.cos(s_lat)*math.cos(e_lat) * math.sin((e_lng - s_lng)/2)**2

    return 2 * R * math.asin(math.sqrt(d))

@cuda.jit
def get_nearby_kernel(coord1, coord2, max_dist, out):
    start = cuda.grid(1)
    stride = cuda.gridsize(1)
    lat_filter = max_dist / 100
    
    for i in range(start, coord1.shape[0], stride):
        ct = 0
        _lat1 = coord1[i,0]
        _lng1 = coord1[i,1]
        
        for j in range(coord2.shape[0]):
            _lat2 = coord2[j,0]
            _lng2 = coord2[j,1]

            if math.fabs(_lat1 - _lat2) <= lat_filter:
                dist = haversine_cuda(_lat1, _lng1, _lat2, _lng2)
                if dist < max_dist:ct += 1
                
        out[i] = ct
        
threads_per_block = 512
blocks_per_grid = 36

In [37]:
coord1_gpu = cuda.to_device(coord1)
coord2_gpu = cuda.to_device(coord2)
out_gpu = cuda.device_array(shape=(n,), dtype=np.int32)

In [38]:
_t_nker = %timeit -o \
get_nearby_kernel[blocks_per_grid, threads_per_block](coord1_gpu, coord2_gpu, 1.0, out_gpu); \
out_gpu.copy_to_host()

1.04 ms ± 200 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
est_time(_t_nker.average)

Примерно 2.9 часов, чтобы перебрать полный датасет
